# mizerShelf Model Hatchery
[mizerShelf model description](https://sizespectrum.org/mizerShelf/articles/model_description.html)

In [ ]:
source("hatchery.R") # imports mizer
library(mizerExperimental)
library(mizerShelf)
library(ggplot2)
library(plotly)

In [ ]:
params <- NWMed_params
params <- steady(params, tol = 1e-3)

target = c("Red mullet", "Striped red mullet", "Hake", "Angler fish", 
           "Poor cod", "Horse mackerel", "Blue whiting",
           "Horned octopus", "Shortfin squid", "Angular crab", "Harbour crab")

plotlySpectra(params, resource = FALSE, species = target, ylim = c(1e-7, NA))
ggsave(sprintf("mizerShelf/spectrumbase.png",t), dpi=300, width = 8, height = 4)


In [ ]:
# Detritus and carrion dynamics
detritus_lifetime(NWMed_params) <- 1
carrion_lifetime(NWMed_params) <- 1/365
carrion_human_origin(NWMed_params) <- 0.2

In [ ]:
# determine more reasonable hatchery input by fraction of total number of mature
# individuals maturity
N_mat <- getN(params, min_w=species_params(params)$w_mat)

# number of individuals to add from hatchery as a % of N_mat, i.e. ratio=1 => 
# add one individual for every mature individual in population
ratio <- 10
#! what is a sensible ratio? is this even a sensible way to scale?

hatchery_target = "Angular crab"
hatchery_params <- data.frame(species = hatchery_target, mu = 0.5, sigma = 0.1, annual_N=N_mat[hatchery_target]*ratio)
hatchery_params

# define new hatchery time dependence function (default is constant 1)
# hatchery_T <- function(t) {
#     return(1 - sign(sin(2 * pi * t))) # square wave
# }

In [ ]:
baseline <- project(params, t_max=15, t_save=0.2, dt=0.05)
hatchery <- projectHatchery(params, t_max=15, t_save=0.2, dt=0.05)

In [ ]:
plotlyBiomassRelative(hatchery, baseline)


In [ ]:
plotlyBiomassRelative(hatchery, baseline, species=target)
ggsave(sprintf("mizerShelf/relbiomass2.png",t), dpi=300, width = 8, height = 4)

Here, the results seem to suggest, rather trivially that most species maintain roughly the same size except for the `hatchery_species`. This may be because the "Angular crab" was chosen as the hatchery species which is lower down the size spectrum than many of the target species, hence it will not affect many of the other species through predation. We do note that the "Horned octopus" sees a ~7% increase in size, this likely suggests that these predate on "Angular crabs" (see check below).

In [ ]:
plotlySpectra(hatchery, resource = FALSE, species = target, ylim = c(1e-7, NA), xlim = c(1e-2,NA))
ggsave(sprintf("mizerShelf/spectrumhatchery.png",t), dpi=300, width = 8, height = 4)

plotlySpectra(params, resource = FALSE, species = target, ylim = c(1e-7, NA), xlim = c(1e-3,NA))

In [ ]:
plotlyPredMort(hatchery, species= target)

In [ ]:
animateSpectra(hatchery)

In [ ]:
#! something not working
# params2 <- getParams(hatchery)
# params2@initial_n <- hatchery@n[length(hatchery@n[,1,1]),,]

# ggplotly(plotDiet(params2,  species="Horned octopus"))

In [ ]:
all(hatchery@n[1,,] == params@initial_n)

In [ ]:
length(hatchery@n[,1,1])

## Shannon Index

The Shannon index (a.k.a. Shannon-Wiener diversity index) provides a metric for species diversity within an ecosystem. It is based on the Shannon entropy which provides a metric for the "amount of information" in a system (applications in information theory).

The Shannon index of an ecological system with species $i \in \mathcal{S}$ is given by
$$
H = - \sum p_i \log(p_i)
$$
where $p_i$ is the proportion of the total number of individuals for species $i$, i.e.
$$
p_i = \frac{n_i}{N}
$$
where $n_i$ is the number of individuals of species $i$ and $N = \sum n_i$ is the total number of individuals.

In [ ]:
plotlySpectra(params, power=0)

In [ ]:
getN(params, min_w = 2.58e-04, max_w = 9.14e+00)["Angular crab"]
getN(params, min_w = 9.14e+00, max_w = 3.75e+01)["Angular crab"]
getN(params)["Angular crab"]

# species_params(params)